# Выбор локации для скважины

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy import stats as st
import matplotlib.pyplot as plt
import warnings; warnings.filterwarnings(action='ignore')

In [ ]:
geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')
display(geo_data_0.head())
display(geo_data_1.head())
display(geo_data_2.head())

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Из представленных строк сложно провести какую-либо аналитику, т.к. значение показателей неизвестно. Известно только что в целевом столбце product указан объем залежей. Посмотрим наполнение таблиц и статистики данных.

In [3]:
display(geo_data_0.info())
display(geo_data_1.info())
display(geo_data_2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

Все данные заполнены, пропусков нет.

In [4]:
display(geo_data_0.describe())
display(geo_data_1.describe())
display(geo_data_2.describe())

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


Для всех датасетов средние близки к медианным значениям, значит значительных выбросов в данных нет.

In [5]:
#Т.к. мы будем использовать эти данные для прогнозирования, то столбцы с id нам не нужны, удалим их из датасетов.
geo_data_0 = geo_data_0.drop(['id'], axis=1)
geo_data_1 = geo_data_1.drop(['id'], axis=1)
geo_data_2 = geo_data_2.drop(['id'], axis=1)

In [6]:
#проверим данные на наличие дубликатов
geo_data_0.duplicated().sum()

0

In [7]:
geo_data_1.duplicated().sum()

0

In [8]:
geo_data_2.duplicated().sum()

0

Дубликатов в данных нет.

Данные загружены и проанализированы, необходимости в серьезной обработке не выявлено.

## Обучение и проверка модели

In [9]:
# Разделим данные для региона 0 на обучающую и валидационную выборки в соотношении 75:25
geo_data_0_train, geo_data_0_valid = train_test_split(geo_data_0, test_size=0.25, random_state=12345)
geo_data_0_train.shape, geo_data_0_valid.shape

((75000, 4), (25000, 4))

In [10]:
features_geo_data_0_train = geo_data_0_train.drop(['product'], axis=1)
target_geo_data_0_train = geo_data_0_train['product']
features_geo_data_0_valid = geo_data_0_valid.drop(['product'], axis=1)
target_geo_data_0_valid = geo_data_0_valid['product']

In [11]:
# Разделим данные для региона 1 на обучающую и валидационную выборки в соотношении 75:25
geo_data_1_train, geo_data_1_valid = train_test_split(geo_data_1, test_size=0.25, random_state=12345)
geo_data_1_train.shape, geo_data_1_valid.shape

((75000, 4), (25000, 4))

In [12]:
features_geo_data_1_train = geo_data_1_train.drop(['product'], axis=1)
target_geo_data_1_train = geo_data_1_train['product']
features_geo_data_1_valid = geo_data_1_valid.drop(['product'], axis=1)
target_geo_data_1_valid = geo_data_1_valid['product']

In [13]:
# Разделим данные для региона 2 на обучающую и валидационную выборки в соотношении 75:25
geo_data_2_train, geo_data_2_valid = train_test_split(geo_data_2, test_size=0.25, random_state=12345)
geo_data_2_train.shape, geo_data_2_valid.shape

((75000, 4), (25000, 4))

In [14]:
features_geo_data_2_train = geo_data_2_train.drop(['product'], axis=1)
target_geo_data_2_train = geo_data_2_train['product']
features_geo_data_2_valid = geo_data_2_valid.drop(['product'], axis=1)
target_geo_data_2_valid = geo_data_2_valid['product']

In [15]:
#Обучение и проверка для региона 0
model_0 = LinearRegression()
model_0.fit(features_geo_data_0_train, target_geo_data_0_train)
predicted_valid_0 = model_0.predict(features_geo_data_0_valid)
print('Средний запас сырья для региона 0:', predicted_valid_0.mean())
print('RMSE для региона 0:', mean_squared_error(target_geo_data_0_valid, predicted_valid_0)**0.5)

Средний запас сырья для региона 0: 92.59256778438035
RMSE для региона 0: 37.5794217150813


In [16]:
print('Фактический средний запас сырья для региона 0:', geo_data_0['product'].mean())
print('Фактическое стандартное отклонение для региона 0:', geo_data_0['product'].std())

Фактический средний запас сырья для региона 0: 92.50000000000001
Фактическое стандартное отклонение для региона 0: 44.288691136853394


In [17]:
#Обучение и проверка для региона 1
model_1 = LinearRegression()
model_1.fit(features_geo_data_1_train, target_geo_data_1_train)
predicted_valid_1 = model_1.predict(features_geo_data_1_valid)
print('Средний запас сырья для региона 1:', predicted_valid_1.mean())
print('RMSE для региона 1:', mean_squared_error(target_geo_data_1_valid, predicted_valid_1)**0.5)

Средний запас сырья для региона 1: 68.728546895446
RMSE для региона 1: 0.893099286775617


In [18]:
print('Фактический средний запас сырья для региона 1:', geo_data_1['product'].mean())
print('Фактическое стандартное отклонение для региона 1:', geo_data_1['product'].std())

Фактический средний запас сырья для региона 1: 68.82500000000002
Фактическое стандартное отклонение для региона 1: 45.94442289497512


In [19]:
#Обучение и проверка для региона 2
model_2 = LinearRegression()
model_2.fit(features_geo_data_2_train, target_geo_data_2_train)
predicted_valid_2 = model_2.predict(features_geo_data_2_valid)
print('Средний запас сырья для региона 2:', predicted_valid_2.mean())
print('RMSE для региона 2:', mean_squared_error(target_geo_data_2_valid, predicted_valid_2)**0.5)

Средний запас сырья для региона 2: 94.96504596800489
RMSE для региона 2: 40.02970873393434


In [20]:
print('Фактический средний запас сырья для региона 2:', geo_data_2['product'].mean())
print('Фактическое стандартное отклонение для региона 2:', geo_data_2['product'].std())

Фактический средний запас сырья для региона 2: 95.00000000000004
Фактическое стандартное отклонение для региона 2: 44.74992106895218


Анализируя полученные модели, можно предположить, что рациональнее добывать нефть в регионе 2, так как именно в этом регионе максимальный средний запас сырья, что говорит о возможности принести наибольшую прибыль.

## Подготовка к расчёту прибыли

In [21]:
#бюджет на разработку скважин в одном регионе в рублях
budget = 10000000000

#количество скважин для бурения
oil_wells_number = 200

#бюджет на одну скважину
budget_for_one = budget / oil_wells_number

#доход с одной единицы продукта в рублях
revenue_rub = 450000

#максимальная доля убытков
max_risk = 0.025

#рублей в млрд
mlrd = 1000000000

Чтобы проект был прибыльным, посчитаем минимальное среднее количество продукта в месторождениях региона, достаточное для разработки.

In [22]:
budget_for_one / revenue_rub

111.11111111111111

Необходимый объем сырья для того, чтобы разработка скважины была окупаемой составил 111 тыс. баррелей. Грубо говоря - это минимальный объем сырья, при котором проект не станет убыточным. Если сравнивать со средним истинным объемом сырья, который мы посчитали в предыдущем шаге, то проект будет убыточным в любом регионе. Но мы будем исследовать только 500 скважин, из которых выберем 200 самых прибыльных.

In [23]:
print("Средний истинный запас сырья в регионах, тыс. баррелей:")
print()
print("Регион 1:", target_geo_data_0_valid.mean())
print("Регион 2:", target_geo_data_1_valid.mean())
print("Регион 3:", target_geo_data_2_valid.mean())

Средний истинный запас сырья в регионах, тыс. баррелей:

Регион 1: 92.07859674082927
Регион 2: 68.72313602435997
Регион 3: 94.88423280885438


In [24]:
#Напишем функцию рассчета прибыли по 200 скважинам каждого региона, из которых получается самый большой объем сырья.
def revenue(target, probabilities):
    target = pd.Series(target).reset_index(drop=True)
    probabilities = pd.Series(probabilities).reset_index(drop=True)
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:oil_wells_number]
    return ((revenue_rub * selected.sum()) - budget) / mlrd

In [25]:
revenue(target_geo_data_0_valid, predicted_valid_0)

3.3208260431398506

In [26]:
revenue(target_geo_data_1_valid, predicted_valid_1)

2.4150866966815108

In [27]:
revenue(target_geo_data_2_valid, predicted_valid_2)

2.7103499635998327

На данном этапе, судя из рассчетов, самым прибыльным является регион 0 - выручка здесь больше 3-х млрд рублей.

## Расчёт прибыли и рисков 

In [28]:
#Посчитаем риски и прибыль для всех трех регионов с помощью процедуры Bootstrap.
state = np.random.RandomState(12345)

In [29]:
def bootstrap (target, predictions):
    values = []
    for i in range(1000):
        target_subsample = target.reset_index(drop=True).sample(n=500, replace=True, random_state=state)
        probs_subsample = predictions[target_subsample.index]
        rev = revenue(target_subsample, probs_subsample)
        values.append(rev)

    values = pd.Series(values)
    lower = values.quantile(max_risk)
    confidence_interval = st.t.interval(0.95, len(values)-1, values.mean(), values.sem())
    mean = values.mean()

    print("Средняя выручка:", mean)
    print("2.5%-квантиль:", lower)
    print("Доверительный интервал:", confidence_interval)
    print("Риски убытка:", (values < 0).mean())

In [30]:
bootstrap(target_geo_data_0_valid, predicted_valid_0)

Средняя выручка: 0.3961649848023711
2.5%-квантиль: -0.11121554589049526
Доверительный интервал: (0.3796203151479725, 0.4127096544567697)
Риски убытка: 0.069


In [31]:
bootstrap(target_geo_data_1_valid, predicted_valid_1)

Средняя выручка: 0.46115581727723964
2.5%-квантиль: 0.07805081075174171
Доверительный интервал: (0.4488289364815785, 0.4734826980729008)
Риски убытка: 0.007


In [32]:
bootstrap(target_geo_data_2_valid, predicted_valid_2)

Средняя выручка: 0.39295047517060455
2.5%-квантиль: -0.11222762537857565
Доверительный интервал: (0.3761642253295495, 0.4097367250116596)
Риски убытка: 0.065


# Вывод
Исходя из полученных результатов, можно сделать предположение, что наиболее перспективным является регион 1, так как обладает наибольшей средней прибылью и не имеет убытков. Стоит отметить, что ни один регион не несет убытки. В итоге, для реализации проекта больше всего подходит регион 1. Бурение скважин здесь связанно с наименьшими рисками и принесет бОльшую выручку.